# Fama-MacBeth Regression #

### Market, Size, Value, Momentum and Beta ###

In [32]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Statistics
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.diagnostic import acorr_ljungbox

# Manipulate Files
import os

# Pretty Notation
from IPython.display import display, Math

In [33]:
# Get the important data for the Risk Free Rate

rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)

# Get the important data for the S&P500

sp500 = pd.read_csv(r"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

In [34]:
# Get the important data for the Betas

# Market Betas
carhart_mkt_betas = pd.read_csv(r"..\additional_data\carhart_mkt_betas.csv")
carhart_mkt_betas.set_index('Date', inplace = True)
carhart_mkt_betas.index = pd.to_datetime(carhart_mkt_betas.index)

# SMB Betas
carhart_smb_betas = pd.read_csv(r"..\additional_data\carhart_smb_betas.csv")
carhart_smb_betas.set_index('Date', inplace = True)
carhart_smb_betas.index = pd.to_datetime(carhart_smb_betas.index)

# HML Betas
carhart_hml_betas = pd.read_csv(r"..\additional_data\carhart_hml_betas.csv")
carhart_hml_betas.set_index('Date', inplace = True)
carhart_hml_betas.index = pd.to_datetime(carhart_hml_betas.index)

# WML Betas
carhart_wml_betas = pd.read_csv(r"..\additional_data\carhart_wml_betas.csv")
carhart_wml_betas.set_index('Date', inplace = True)
carhart_wml_betas.index = pd.to_datetime(carhart_wml_betas.index)

# AMD Betas
carhart_amd_betas = pd.read_csv(r"..\additional_data\carhart_amd_betas.csv")
carhart_amd_betas.set_index('Date', inplace = True)
carhart_amd_betas.index = pd.to_datetime(carhart_amd_betas.index)

In [35]:
# Dictionary to store the DataFrames
folder_path = r"..\stocks"

dataframes = {}

# List all files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        # Full path to the file
        file_path = os.path.join(folder_path, file)
        
        # Read the Excel file
        df = pd.read_csv(file_path)
        df = df.set_index("Date")
        df.index = pd.to_datetime(df.index)

        df = df[['Adjusted_close', 'Company Market Cap']]

        df = df.rename(columns={
            'Adjusted_close':'adj_close',
            'Company Market Cap':'market_cap',
        })

        # Fill nans
        df['adj_close'] = df['adj_close'].interpolate(method='time')
        df['market_cap'] = df['market_cap'].interpolate(method='time')

        df = df.loc['2015-01-01':]

        df.dropna(inplace=True)
        
        if len(df) >= 2000:
            # File name without extension
            file_name = os.path.splitext(file)[0]
            
            # Guardar en el diccionario
            dataframes[file_name] = df
            print(f"File loaded: {file_name} ({len(df)} rows)")
        else:
            print(f"File skipped (less than 2000 rows after cleaning): {file}")

print(f"\nTotal files loaded: {len(dataframes)}")
print("Files loaded:", list(dataframes.keys()))

In [36]:
# Create a DataFrame
mkt_cap_dict = {}

# Recorrer cada ticker y DataFrame
for ticker, df in dataframes.items():
    
    mkt_cap_dict[ticker] = df['market_cap']

mkt_cap_df = pd.DataFrame.from_dict(mkt_cap_dict)

mkt_cap_df

In [37]:
# Let us obtain the betas of each stock

returns_dic = {}

for ticker, df in dataframes.items():
    
    returns_dic[ticker] = df['adj_close'].pct_change(1)

returns_df = pd.DataFrame.from_dict(returns_dic)
returns_df = returns_df.apply(lambda x: x.fillna(x.mean()), axis=0)

returns_df

In [38]:
# Intersec Dates
common_dates = returns_df.index.intersection(carhart_mkt_betas.index)

# Filter for common dates
mkt_cap_df = mkt_cap_df.loc[common_dates]
returns_df = returns_df.loc[common_dates]
carhart_mkt_betas = carhart_mkt_betas.loc[common_dates]
carhart_smb_betas = carhart_smb_betas.loc[common_dates]
carhart_hml_betas = carhart_hml_betas.loc[common_dates]
carhart_wml_betas = carhart_wml_betas.loc[common_dates]
carhart_amd_betas = carhart_amd_betas.loc[common_dates]

In [39]:
# Initialize lists to store results
betas_list = []

# Loop over each available date in the returns DataFrame
for date in common_dates:
    # Cross-section of market caps, returns, and betas for that specific date
    weights = np.sqrt(mkt_cap_df.loc[date])
    weights = weights / weights.sum()

    # Returns
    rets = pd.DataFrame([returns_df.loc[date]], index=['returns']).transpose()

    # The Model Betas
    mkt = pd.DataFrame([carhart_mkt_betas.loc[date]], index=['mkt']).transpose()
    smb = pd.DataFrame([carhart_smb_betas.loc[date]], index=['smb']).transpose()
    hml = pd.DataFrame([carhart_hml_betas.loc[date]], index=['hml']).transpose()
    wml = pd.DataFrame([carhart_wml_betas.loc[date]], index=['wml']).transpose()
    amd = pd.DataFrame([carhart_amd_betas.loc[date]], index=['amd']).transpose()
    
    # Merge returns with characteristics
    reg_df_data = pd.concat([rets, mkt, smb, hml, wml, amd], axis=1).dropna()

    # Define independent (X) and dependent (y) variables
    X = sm.add_constant(reg_df_data[['mkt', 'smb', 'hml', 'wml', 'amd']])  
    y = reg_df_data['returns']  

    # Run the weighted least squares (WLS) regression
    model = sm.WLS(y, X, missing='drop', weights=weights)
    results = model.fit()

    # Extract coefficients and p-values
    params = results.params
    #pvalues = results.pvalues
    
    # Set non-significant betas to zero (e.g., p > 0.05)
    #significance_level = 0.1
    #params[pvalues > significance_level] = 0
    
    # Store results
    betas_list.append(pd.Series(params, name=date))

# Convert lists to DataFrames
history_betas_df = pd.DataFrame(betas_list)

# Set the index as the dates
history_betas_df.index = common_dates

In [40]:
# Check the DataFrames

history_betas_df

In [41]:
# Create the Plot
plt.figure(figsize=(10, 6))
plt.plot(history_betas_df['mkt'].cumsum(), label='Market Beta Returns', alpha=0.7)
plt.plot(history_betas_df['smb'].cumsum(), label='SMB Beta Returns', alpha=0.7)
plt.plot(history_betas_df['hml'].cumsum(), label='HML Beta Returns', alpha=0.7)
plt.plot(history_betas_df['wml'].cumsum(), label='WML Beta Returns', alpha=0.7)
plt.plot(history_betas_df['amd'].cumsum(), label='AMD Beta Returns', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Factor Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()

# Show
plt.show()

In [42]:
# Lets test the significance of these coefficients
def newey_west_std(errors, lag=4):
    """
    Computes Newey-West standard errors for a time series.
    
    Parameters:
    errors: Pandas Series or NumPy array of residuals (gamma estimates)
    lag: Maximum number of lags to consider (default: 4)
    
    Returns:
    Newey-West adjusted standard error
    """
    T = len(errors)
    gamma_var = errors.var()  # Start with variance of the series
    
    for l in range(1, lag + 1):
        weight = 1 - (l / (lag + 1))
        autocov = np.cov(errors[:-l], errors[l:])[0, 1]  # Autocovariance at lag l
        gamma_var += 2 * weight * autocov  # Newey-West adjustment

    return np.sqrt(gamma_var / T)  # Standard error

def fama_macbeth_significance_test(gamma_series, lag=4):
    """
    Performs statistical significance tests for Fama-MacBeth risk premia.

    Parameters:
    gamma_series: DataFrame where each column contains estimated gammas over time.
    lag: Lags for Newey-West standard errors (default: 4).

    Returns:
    DataFrame with mean gamma, standard error, t-statistics, and p-values.
    """
    gamma_means = gamma_series.mean()

    # Compute Newey-West adjusted standard errors
    gamma_std = gamma_series.apply(newey_west_std, lag=lag)

    # Compute t-statistics
    t_stats = gamma_means / gamma_std

    # Compute p-values
    p_values = 2 * (1 - stats.t.cdf(abs(t_stats), df=len(gamma_series) - 1))

    # Create results DataFrame
    results = pd.DataFrame({
        'Mean Gamma': gamma_means,
        'Std Error': gamma_std,
        't-stat': t_stats,
        'p-value': p_values
    })

    return results

In [43]:
# Now the Results

results = fama_macbeth_significance_test(history_betas_df[['mkt', 'smb', 'hml', 'wml', 'amd']])

results